In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Loading inflation data 


if_df = pd.read_csv('scraped_cpi.csv')

In [6]:
print(if_df.columns)

Index(['base_year', 'series', 'year', 'month', 'state', 'sector', 'division',
       'group', 'class', 'sub_class', 'item', 'code', 'index', 'inflation',
       'imputation'],
      dtype='object')


In [7]:
def calculate_mom_change(df, value_column='index', group_columns=None, new_column_name='mom_change'):
    """
    Calculate month-on-month percentage change for inflation data.

    Parameters:
    df (pd.DataFrame): Input dataframe with inflation data
    value_column (str): Column name to calculate change on (default: 'index')
    group_columns (list): Columns to group by for unique series (default: hierarchical categories)
    new_column_name (str): Name for the new MoM change column (default: 'mom_change')

    Returns:
    pd.DataFrame: DataFrame with added MoM change column
    """
    if group_columns is None:
        group_columns = ['state', 'sector', 'division', 'group', 'class', 'sub_class', 'item', 'code']

    # Sort the dataframe by group columns and time
    sort_columns = group_columns + ['year', 'month']
    df_sorted = df.sort_values(sort_columns).copy()

    # Calculate percentage change within each group
    df_sorted[new_column_name] = df_sorted.groupby(group_columns)[value_column].pct_change() * 100

    return df_sorted

In [8]:
# Apply month-on-month change calculation
if_df_mom = calculate_mom_change(if_df, value_column='index')

# Display sample of the results
print("Sample of data with MoM change:")
print(if_df_mom[['state', 'item', 'year', 'month', 'index', 'mom_change']].head(10))

Sample of data with MoM change:
          state item  year     month   index  mom_change
2672  All India    *  2025     April  101.58         NaN
5344  All India    *  2025    August  103.74    2.126403
8016  All India    *  2025  December  104.10    0.347021
1336  All India    *  2025  February  101.32   -2.670509
668   All India    *  2025   January  101.67    0.345440
4676  All India    *  2025      July  103.35    1.652405
4696  All India    *  2025      July  103.35    0.000000
4716  All India    *  2025      July  103.35    0.000000
4008  All India    *  2025      June  102.51   -0.812772
2004  All India    *  2025     March  101.39   -1.092576
